In [0]:
import pytest
from pyspark.sql import SparkSession
from pyspark.sql.types import (
    IntegerType,
    DoubleType,
    StringType,
    DateType,
    StructType,
    StructField,
)
from pyspark.sql.functions import *

In [0]:
%run ./common_functions

In [0]:
# Create a SparkSession
spark = SparkSession.builder.appName("PEI_SD_Assessment_unit_testing").getOrCreate()

In [0]:
# unit test case for format columns
def test_format_column_names(spark):
    # Sample input data
    data = [
        {
            "Row ID": 1,
            "Order ID": "CA-2016-122581",
            "Order Date": "2016-08-21",
            "Ship Date": "2016-08-25",
            "Ship Mode": "Standard Class",
            "Customer ID": "JK-15370",
            "Product ID": "FUR-CH-10002961",
            "Quantity": 7,
            "Price": 573.174,
            "Discount": 0.3,
            "Profit": 63.686,
        }
    ]

    df_order = spark.createDataFrame(data)

    formatted_df = format_column_names(df_order)
    expected_columns = [
        "Customer_ID",
        "Discount",
        "Order_Date",
        "Order_ID",
        "Price",
        "Product_ID",
        "Profit",
        "Quantity",
        "Row_ID",
        "Ship_Date",
        "Ship_Mode",
    ]
    assert (
        formatted_df.columns == expected_columns
    ), "Columns are not formatted correctly"

    return formatted_df


test_format_column_names(spark)

DataFrame[Customer_ID: string, Discount: double, Order_Date: string, Order_ID: string, Price: double, Product_ID: string, Profit: double, Quantity: bigint, Row_ID: bigint, Ship_Date: string, Ship_Mode: string]

In [0]:
# unit test case for extracting_price
def test_orders_extract_double(spark):
    # Sample input data
    data = [
        Row(price="$1,234.56"),
        Row(price="USD 567.89"),
        Row(price="€89"),
        Row(price="Free"),
        Row(price=None),
        Row(price="1,000"),
        Row(price="42.0 dollars"),
    ]
    df = spark.createDataFrame(data)

    # Apply the price extraction function
    result_df = orders_extract_double(df, "price", "price")

    # Collect result as list
    result = [row["price"] for row in result_df.select("price").collect()]

    # Expected results
    expected = [1234.56, 567.89, 89.0, None, None, 1000.0, 42.0]

    # Assert
    assert result == expected


test_orders_extract_double(spark)

In [0]:
# unit test for casting columns
def test_cast_columns(spark):
    # Sample input data (all as strings initially)
    data = [
        {
            "Row ID": "1",
            "Order ID": "CA-2016-122581",
            "Order Date": "21/8/2016",
            "Ship Date": "25/8/2016",
            "Ship Mode": "Standard Class",
            "Customer ID": "JK-15370",
            "Product ID": "FUR-CH-10002961",
            "Quantity": "7",
            "Price": "573.174",
            "Discount": "0.3",
            "Profit": "63.686",
        }
    ]

    df = spark.createDataFrame(data)

    # Define column type mapping
    cast_map = {
        "Row ID": IntegerType(),
        "Order ID": StringType(),
        "Order Date": DateType(),
        "Ship Date": DateType(),
        "Ship Mode": StringType(),
        "Customer ID": StringType(),
        "Product ID": StringType(),
        "Quantity": IntegerType(),
        "Price": DoubleType(),
        "Discount": DoubleType(),
        "Profit": DoubleType(),
    }

    # Date format mapping for date columns
    date_format_map = {"Order Date": "d/M/yyyy", "Ship Date": "d/M/yyyy"}

    # Apply casting
    casted_df = cast_columns(df, cast_map, date_format_map)

    # Check schema types
    schema_types = {field.name: field.dataType for field in casted_df.schema.fields}
    assert isinstance(schema_types["Row ID"], IntegerType)
    assert isinstance(schema_types["Order Date"], DateType)
    assert isinstance(schema_types["Price"], DoubleType)
    assert isinstance(schema_types["Discount"], DoubleType)

    # Validate values
    row = casted_df.collect()[0]
    assert row["Row ID"] == 1
    assert row["Quantity"] == 7
    assert row["Price"] == 573.174
    assert row["Discount"] == 0.3
    assert row["Order Date"].strftime("%Y-%m-%d") == "2016-08-21"
    assert row["Ship Date"].strftime("%Y-%m-%d") == "2016-08-25"


test_cast_columns(spark)

In [0]:
# unit test case to clean the name
def test_clean_name():
    data = [("  john   doe123 ",), ("Ann3a## Bell",), ("  aLiCe   ",), ("",), (None,)]
    schema = StructType([StructField("name", StringType(), True)])
    raw_name_df = spark.createDataFrame(data, schema)
    result_df = clean_name(raw_name_df)
    results = [row["name"] for row in result_df.collect()]
    expected = ["John Doe", "Anna Bell", "Alice", "", None]

    assert results == expected


test_clean_name()

In [0]:
# Unit test case for phone_df
def test_clean_phone_numbers(spark):
    data = [
        ("421.580.0902x9815",),  # valid after cleaning
        ("001-542-415-0246x314",),  # valid after stripping 001
        ("7185624866",),  # already clean
        ("265.101.5569x1098",),  # valid
        ("001-473-645-2141x9154",),  # valid after stripping
        ("(563)647-4830x5318",),  # valid
        ("-6181",),  # invalid
        ("067.435.8553x692",),  # not 10 digits after cleaning
        ("786.638.6820",),  # valid
        ("3528465094",),  # valid
        ("(722)376-5599x0693",),  # valid
        ("817.409.0760x92195",),  # valid
        ("924.484.7935",),  # valid
    ]
    schema = StructType([StructField("phone", StringType(), True)])
    phone_df = spark.createDataFrame(data, schema)
    cleaned_df = clean_phone_numbers(phone_df, "phone")
    actual = [row["phone"] for row in cleaned_df.collect()]

    expected = [
        "4215800902",
        "5424150246",
        "7185624866",
        "2651015569",
        "4736452141",
        "5636474830",
        None,
        "0674358553",
        "7866386820",
        "3528465094",
        "7223765599",
        "8174090760",
        "9244847935",
    ]

    assert actual == expected


# Create a Spark session
spark = SparkSession.builder.appName("PhoneCleaningTest").getOrCreate()

# Call the test function with the Spark session
test_clean_phone_numbers(spark)

In [0]:
# unit test case for email check
def test_clean_email_column(spark):
    schema = StructType([StructField("email", StringType(), True)])
    data = [
        (" alejandrobailey724@gmail.com ",),  # with whitespace
        ("DANIELPAUL555@GMAIL.COM",),  # uppercase
        ("jamesjohnson660@gmail.com",),  # valid
        ("sabrinahayes@@gmail.com",),  # invalid (double @)
        ("bad_email.com",),  # invalid
        ("william+test@domain.org",),  # valid
        ("test@sub.domain.com",),  # valid
        ("user@invalid_domain",),  # invalid
        ("",),  # empty
        (None,),  # null
    ]
    email_df = spark.createDataFrame(data, schema)
    cleaned_df = clean_email_column(email_df, "email")
    actual = [row["email"] for row in cleaned_df.collect()]
    expected = [
        "alejandrobailey724@gmail.com",
        "danielpaul555@gmail.com",
        "jamesjohnson660@gmail.com",
        None,  # sabrinahayes@@gmail.com
        None,  # bad_email.com
        "william+test@domain.org",
        "test@sub.domain.com",
        None,  # user@invalid_domain
        None,  # empty string
        None,  # null
    ]
    assert actual == expected


test_clean_email_column(spark)

In [0]:
# unit test to create delta table
def test_create_and_verify_delta_table(spark):
    data = [("Alice", 30), ("Bob", 40)]
    schema = StructType(
        [
            StructField("name", StringType(), True),
            StructField("age", IntegerType(), True),
        ]
    )
    df = spark.createDataFrame(data, schema)

    # Clean up if test_table exists
    spark.sql("DROP TABLE IF EXISTS test_db.test_table")
    spark.sql("CREATE DATABASE IF NOT EXISTS test_db")

    result = create_and_verify_delta_table(
        spark=spark,
        df=df,
        catalog_name="test_catalog",
        table_name="test_table",
        database="test_db",
        mode="overwrite",
        drop_if_exists=True,
        merge_schema=True,
    )

    assert result is True

    # Verify table data
    df2 = spark.table("test_catalog.test_db.test_table")
    assert df2.count() == 2
    assert set(df2.columns) == {"name", "age"}


test_create_and_verify_delta_table(spark)